In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

class Annonce:
    def __init__(self, idAnnonce, version, annee, kilometrage, prix, telephone, isPro, description):
        self.idAnnonce = idAnnonce
        self.version = version
        self.annee = annee
        self.kilometrage = kilometrage
        self.prix = prix
        self.telephone = telephone
        self.isPro = isPro
        self.description = description
    
    def to_dict(self):
        return {
            #'idAnnonce': self.idAnnonce,
            'annee': self.annee,
            'kilometrage':self.kilometrage, 
            'prix':self.prix,
            'version':self.version,
            'isPro':self.isPro, 
            'telephone':self.telephone
            #'description':self.description
            
        }

def getSoupFromURL(url, method='get', data={}):

    if method == 'get':
        res = requests.get(url)
    elif method == 'post':
        res = requests.post(url, data=data)
    else:
        return None

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    else:
        return None

def getVersion(descr):
    m = re.search("ZOE (\w+)", descr)
    if m == None:
        return ""
    else:
        return m.group(1)

def prettifyPrix(prix):
    prix = prix.replace(" ","").replace ("€","").strip()
    return prix

def prettifyKm(km):
    km = km.replace(" ","").replace ("KM","").strip()
    return km

def getPhone(descr):
    expression = r"0[0-9]([ .-]?[0-9]{2}){4}"
    m = re.search(expression, descr)
    if m == None:
        return ""
    else:
        return m.group(0)
    
def getDetail(urlItem, cpt):
    annonce = Annonce(0,0,0,0,0,0,0,0)
    if urlItem != None:
        soupItem = getSoupFromURL(urlItem)
        if soupItem:
            b = soupItem.find_all(class_="line")
            for b0 in b:
                
                b1 = b0.find_all(class_="property")
                b1Prime = b0.find_all(class_="value")
                  
                for b2 in b1:
                    for  b2Prime in b1Prime:
                        annonce.idAnnonce = cpt
                        if (b2.text).upper() == "PRIX":
                            annonce.prix = prettifyPrix(b2Prime.text)
                        if (b2.text).upper() == "ANNÉE-MODÈLE":
                            annonce.annee = b2Prime.text.strip()
                        if (b2.text).upper() == "KILOMÉTRAGE":
                            annonce.kilometrage = prettifyKm(b2Prime.text)
                        if (b2.text).upper() == "DESCRIPTION :":
                            annonce.description = b2Prime.text.upper()
                            annonce.version = getVersion(annonce.description)
                            annonce.telephone = getPhone(annonce.description)
                        
                                               
    return annonce

                    
def getInfoVoitures(url):
    list = []
    soup = getSoupFromURL(url)
    if soup:
        a = soup.find_all(class_="tabsContent")
        for  a0 in a:
            a1 = a0.find_all("li")
            cpt=0
            for a2 in a1:
                urlItem = "http:" + a2.find ("a")["href"]
                
                annonce = getDetail(urlItem, cpt)  
                
                if a2.find(class_="ispro")!= None:
                    annonce.isPro = a2.find(class_="ispro").text.replace("(","").replace(")","")
                else:
                    annonce.isPro = "NonPro"
                cpt=cpt+1
                list.append(annonce)
            
    else:
        0
    return list

# essayer de paralleliser !!!

i = 1
df = pd.DataFrame()
while True :
    print(i)
    url="https://www.leboncoin.fr/voitures/offres/ile_de_france/?o="+str(i)+"&q=renault%20zoe"
    i=i+1
    
    try :
        list = getInfoVoitures(url)
        if len(list) > 0:
            df = df.append(pd.DataFrame.from_records([s.to_dict() for s in list]))
        else:
            break
    except StopIteration :
        break

df.to_csv(r'bonCoin.csv', header=["Annee","pro","KM","Prix","Telephone","Version"], index=None, sep=',', mode='a')



1
2
3
4
